In [ ]:
import pandas as pd
import requests
import math
import yfinance as yf
from termcolor import colored as cl
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use("fivethirtyeight")

In [ ]:
aapl = yf.download("AAPL", start="2015-01-01", end="2020-01-01").reset_index()

In [ ]:
# Calculate the ROI
buy_price = aapl["Close"].iloc[0]
sell_price = aapl["Close"].iloc[-1]

roi = (sell_price - buy_price) / buy_price * 100

print(f"The ROI for a buy and hold strategy is {roi}%")

In [ ]:
counter = 1
for period in [5, 8, 13]:
    aapl["SMA" + str(counter)] = aapl["Close"].rolling(window=period).mean()
    counter += 1

In [ ]:
aapl

In [ ]:
plt.plot(aapl["Close"][:100], label="AAPL", linewidth=6)
plt.plot(aapl["SMA5"][:100], label="SMA 5", linewidth=3, linestyle="--")
plt.plot(aapl["SMA8"][:100], label="SMA 8", linewidth=4, alpha=0.7)
plt.plot(aapl["SMA13"][:100], label="SMA 13", linewidth=4, alpha=0.7)
plt.title("AAPL SMA 5,8,13")
plt.legend()

In [ ]:
def implement_strategy(aapl, investment):

    in_position = False
    equity = investment

    for i in range(1, len(aapl)):
        if (
            aapl["SMA8"][i - 1] < aapl["SMA13"][i - 1]
            and aapl["SMA8"][i] > aapl["SMA13"][i]
            and aapl["SMA5"][i] > aapl["SMA8"][i]
            and aapl["Close"][i] > aapl["SMA5"][i]
            and in_position == False
        ):
            no_of_shares = math.floor(equity / aapl.Close[i])
            equity -= no_of_shares * aapl.Close[i]
            in_position = True
            print(
                cl("BUY: ", color="green", attrs=["bold"]),
                f'{no_of_shares} Shares are bought at ${aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
            )
        elif (
            aapl["SMA8"][i - 1] > aapl["SMA13"][i - 1]
            and aapl["SMA8"][i] < aapl["SMA13"][i]
            and aapl["SMA5"][i] < aapl["SMA8"][i]
            and aapl["Close"][i] < aapl["SMA5"][i]
            and in_position == True
        ):
            equity += no_of_shares * aapl.Close[i]
            in_position = False
            print(
                cl("SELL: ", color="red", attrs=["bold"]),
                f'{no_of_shares} Shares are bought at ${aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
            )
    if in_position == True:
        equity += no_of_shares * aapl.Close[i]
        print(
            cl(
                f'\nClosing position at {aapl.Close[i]} on {str(aapl["Date"][i])[:10]}',
                attrs=["bold"],
            )
        )
        in_position = False

    earning = round(equity - investment, 2)
    roi = round(earning / investment * 100, 2)
    print(cl(f"EARNING: ${earning} ; ROI: {roi}%", attrs=["bold"]))

In [ ]:
import plotly.graph_objects as go

# Create a trace for each line
trace1 = go.Scatter(
    x=aapl.index[:100], y=aapl["Close"][:100], mode="lines", name="AAPL"
)
trace2 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA5"][:100],
    mode="lines",
    name="SMA 5",
    line=dict(dash="dash"),
)
trace3 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA8"][:100],
    mode="lines",
    name="SMA 8",
    line=dict(width=4),
)
trace4 = go.Scatter(
    x=aapl.index[:100],
    y=aapl["SMA13"][:100],
    mode="lines",
    name="SMA 13",
    line=dict(width=4),
)

# Create a layout
layout = go.Layout(title="AAPL SMA 5,8,13")

# Create a figure and add the traces
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
layout = go.Layout(
    title="AAPL SMA 5,8,13",
    plot_bgcolor="rgb(230, 230,230)",  # this changes the plot area background color
    paper_bgcolor="rgb(255, 255, 255)",  # this changes the rest of the plot background color
)
fig.update_layout(layout)
# Show the figure
fig.show()

In [ ]:
implement_strategy(aapl, 10000)